![](./img/gumps_logo.PNG)

## or... How I Learned to Stop Worrying and Love Python

## What is GUMPS?

* Generic Universal Modeling Platform Software
* Re-Envision Complex Models as smaller modules and their interfaces
    * Reduces Code Duplication
    * Speeds Up Development


## Kernel
A Basic Unit of Computation, to create one we need to define three things:

1) get_state_class

Defines all of the variables needed for computation.

2) initialize  

A location to create "expensive" objects needed for computation, (loading a neural network)

3) user_defined_function  

Computation code for a single "step", takes a variable registry(attr dataclass) as an argument

In [1]:
from typing import Type
from gumps.kernels import AbstractKernel
import time
import attrs

@attrs.define
class SphereState:
    "states for the SphereKernel"
    n: int
    x_0: float
    x_1: float
    x_2: float
    x_3: float
    total: float = 0.0


class SphereKernel(AbstractKernel):
    """This is a sphere kernel. It is a trivial nd optimization problem """

    def user_defined_function(self, variables:SphereState):
        total = 0.0
        for i in range(variables.n):
            total += (getattr(variables, f'x_{i}') - self.cache[f'a_{i}'])**2.0
        variables.total = total

    def initialize(self):
        #time.sleep(5) #expensive calculation
        n = self.model_variables['n']
        for i in range(n):
            self.cache[f'a_{i}'] = self.model_variables[f'a_{i}']

    def get_state_class(self) -> SphereState:
        return SphereState

#Should not be modified after initialization
model_variables = {'n':4, 'a_0': 0.1, 'a_1':0.2, 'a_2': 0.3, 'a_3': -0.2}

sphere_kernel = SphereKernel(model_variables=model_variables)

#Inspect its allowable states:
print(sphere_kernel.allowed_state)

{'x_2', 'n', 'x_0', 'total', 'x_1', 'x_3'}


## Study

In [2]:
from gumps.studies import SimpleSimulationStudy, SimulationStudy

problem = {'x_0': 1.5, 'x_1':1.6, 'x_2': 0.7, 'x_3': 1.1, 'n':4}
study = SimpleSimulationStudy(problem, sphere_kernel)
result = study.run()
print(result)
print('********')
print(study.state_frame())

SphereState(n=4, x_0=1.5, x_1=1.6, x_2=0.7, x_3=1.1, total=5.7700000000000005)
********
   n  x_0  x_1  x_2  x_3  total
0  4  1.5  1.6  0.7  1.1   5.77


In [3]:
#It throws smart errors!
problem = {'x_0': 1.5, 'x_7':1.6, 'x_2': 0.7, 'x_3': 1.1}
study = SimpleSimulationStudy(problem, sphere_kernel)

IllDefinedException: ('Missing ', {'x_7'})

# Why?

In [4]:
from gumps.studies.batch_study import AbstractBatchStudy
from gumps.common.parallel import Parallel
import pandas as pd
from typing import Callable

class BatchStudyMultiProcess(AbstractBatchStudy):
    "create a batch study by using a SimulationStudy and pool processing"

    def __init__(self, *, study:SimulationStudy, parallel:Parallel):
        "create a simulation study with a parallel pool"
        self.study = study
        self.parallel = parallel

    def start(self):
        "handle any initialization tasks that are needed"
        self.parallel.start()

    def stop(self):
        "handle any shutdown tasks that are needed"
        self.parallel.stop()

    def run(self, input_data: pd.DataFrame, processing_function:Callable) -> pd.DataFrame:
        "run the batch simulation"
        rows = (row.to_dict() for idx,row in input_data.iterrows())
        results = list(self.parallel.runner(self.study.run_data, rows))
        self.save_results(input_data, results)

        return pd.DataFrame(processing_function(result) for result in results)


In [5]:
parallel = Parallel(poolsize=1) #blame jupyter!
batchSphere = BatchStudyMultiProcess(study=study, parallel=parallel)

input_data = pd.DataFrame([
    {'x_0': 1.1, 'x_1':1.2, 'x_2': 0.5, 'x_3': 1.3},
    {'x_0': 1.2, 'x_1':1.3, 'x_2': 0.6, 'x_3': 1.4},
    {'x_0': 1.3, 'x_1':1.5, 'x_2': 0.4, 'x_3': 1.5},
    {'x_0': 1.4, 'x_1':1.1, 'x_2': 0.3, 'x_3': 1.6}
])

#Specify what we want!
def get_total(frame:pd.DataFrame):
    return {'total': frame.total[0]}

with batchSphere:
    totals = batchSphere.run(input_data, get_total)
print(totals)

   total
0   4.29
1   5.07
2   6.03
3   5.74


In [6]:
def get_whatever(frame: pd.DataFrame):
    return {'x0': frame.x_0[0], 'total_squared' : frame.total[0]**2}

with batchSphere:
    totals = batchSphere.run(input_data, get_whatever)

print(totals)

    x0  total_squared
0  1.1        18.4041
1  1.2        25.7049
2  1.3        36.3609
3  1.4        32.9476


In [7]:
import scipy
import gumps.studies.batch_sphere_study
import gumps.solvers.monte_carlo_solver
import gumps.apps.monte_carlo

distributions = {'x_0':scipy.stats.uniform(0.0, 1), 'x_1':scipy.stats.norm(0, 1),
                'x_2':scipy.stats.uniform(-1, 2), 'x_3':scipy.stats.norm(1, 1e-2)}

parameters = gumps.solvers.monte_carlo_solver.MonteCarloParameters(variable_distributions=distributions, target_probability=[0.05],
        window=10, tolerance=1e-2, min_steps=3, sampler_seed=0, sampler_scramble=False)

model_variables = {'a_0': 0.0, 'a_1':0.0, 'a_2':0, 'a_3':0}
diffs = [var.replace('a', 'd') for var in model_variables]

batch = gumps.studies.batch_sphere_study.BatchLineStudy(model_variables=model_variables)

def processing_function(frame: pd.DataFrame):
    "process the dataframe for the loss function"
    return pd.DataFrame(frame[diffs])

app = gumps.apps.monte_carlo.MonteCarloApp(parameters=parameters,
    processing_function=processing_function,
    directory=None,
    batch=batch)
app.run()

answer = app.answer().to_numpy()
print(answer)


[ 0.05043781 -1.64062427 -0.89912439  0.98359376]


In [8]:
import gumps.solvers.sampler
import gumps.apps.parametric_sweep

parameters = gumps.solvers.sampler.SamplerSolverParameters(
            number_of_samples = 10,
            lower_bound = {'x_1':1, 'x_2':2, 'x_3':3},
            upper_bound = {'x_1':5, 'x_2':6, 'x_3':7},
            sampler = "sobol"
            )
model_variables = {'n':4, 'a_0': 0.1, 'a_1':0.2, 'a_2': 0.3, 'a_3': -0.2}

app = gumps.apps.parametric_sweep.ParametricSweepApp(parameters=parameters,
    processing_function=get_total,
    directory=None,
    batch=batchSphere)
app.run()
print(app.responses)

     total
0  15.7300
1  50.5300
2  41.3300
3  65.7300
4  49.9800
5  61.1800
6  62.5800
7  43.3800
8  61.3675
9  63.5675


/home/wheymann/mambaforge/envs/gumps_311/lib/python3.11/site-packages/scipy/stats/_qmc.py:804: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  sample = self._random(n, workers=workers)
